In [1]:
from VwPipeline.Core import Workspace
from VwPipeline import Logger

logger = Logger.console_logger(0, 'DEBUG')
context = Workspace(r'c:\data\sweep\ws', logger)

In [2]:
def generate_something(value1, value2):
    return {'key1': value1, "key2": value2}

In [3]:
context.run(generate_something, 1, 7)
#pseudo-code
#   path = md5("generate_somehting" + "1" + "2")
#   if not path.exists:
#       result = generate_something(1, 2)
#       json.dump(result, path)
#       return result
#   else:
#       return json.read(path)

[0][21-07-2020 10:41:55] Generating path: (generate_something,1,7)	832988cd2b878fc04b9b73a09df422c4
[0][21-07-2020 10:41:55] Result of generate_something is found


{'key1': 1, 'key2': 7}

In [4]:
vw_path = r'C:\src\VowpalWabbit\vowpal_wabbit\vowpalwabbit\out\target\Release\x64\vw.exe'
from VwPipeline.Vw import Vw, VwInput

vw = Vw(vw_path, context, procs = 1)

In [6]:
opts = {'#cmd': '--ccb_explore_adf --epsilon 0.1 --dsjson', '-d': r'C:\data\sweep\1.json'}
vw.run(opts)

[0][21-07-2020 10:42:08] Generating path: (__run__,C:\src\VowpalWabbit\vowpal_wabbit\vowpalwabbit\out\target\Release\x64\vw.exe --ccb_explore_adf --epsilon 0.1 --dsjson -d C:\data\sweep\1.json)	f85a8b0eadcef72d445acea669ee7b5b
[0][21-07-2020 10:42:08] Result of __run__ is found


{'loss_per_example': {'1': '0.000000',
  '2': '0.000000',
  '4': '0.000000',
  '8': '-0.302696',
  '16': '-0.289653',
  '32': '-0.335542',
  '64': '-0.341546',
  '128': '-0.317071',
  '256': '-0.367942',
  '512': '-0.411241',
  '1024': '-0.387492',
  '2048': '-0.400077',
  '4096': '-0.441653',
  '8192': '-0.458337',
  '16384': '-0.463979',
  '32768': '-0.496472',
  '65536': '-0.493486',
  '131072': '-0.494253',
  '262144': '-0.494526',
  '524288': '-0.499413'},
 'since_last': {'1': '0.000000',
  '2': '0.000000',
  '4': '0.000000',
  '8': '-0.605392',
  '16': '-0.276611',
  '32': '-0.381430',
  '64': '-0.347551',
  '128': '-0.292596',
  '256': '-0.418812',
  '512': '-0.454540',
  '1024': '-0.363744',
  '2048': '-0.412662',
  '4096': '-0.483229',
  '8192': '-0.475020',
  '16384': '-0.469621',
  '32768': '-0.528964',
  '65536': '-0.490501',
  '131072': '-0.495021',
  '262144': '-0.494798',
  '524288': '-0.504301'},
 'loss': -0.499472}

In [ ]:
inputs = [r'C:\data\sweep\1.json', r'C:\data\sweep\2.json']

opts_in = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson'}
opts_out = ['--cache_file']

result = vw.test(inputs, opts_in, opts_out)
#pseudocode:
#    for i in inputs:
#       opts_in['-d'] = i
#       populated = populate(opts_in, opts_out)
#       yield return (vw.run(opts_in, populated), populated)

logger.info('Done')
print(result.Loss)
print(result.Populated)

In [ ]:
inputs = [r'C:\data\sweep\2.json', r'C:\data\sweep\3.json']

opts_in = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume'}
opts_out = ['-f']

result = vw.train(inputs, opts_in, opts_out)
#pseudocode:
#    for i in inputs:
#       opts_in['-d'] = i
#       populated = populate(opts_in, opts_out)
#       yield return (vw.run(opts_in, populated), populated)
#       opts_in['-i'] = populated['-f']

logger.info('Done')
print(result.Loss)
print(result.Populated)

In [ ]:
opts_in_1 = {'#base': '--ccb_explore_adf --epsilon 0.1 --dsjson --save_resume'}
opts_in_2 = {'#base': '--ccb_explore_adf --epsilon 0.2 --dsjson --save_resume'}
opts_out = ['-f']

results_par = vw.train(inputs, [opts_in_1, opts_in_2], opts_out)
results_par

In [ ]:
from VwPipeline.VwSweep import VwSweep
from VwPipeline import VwOptsGrid

sweep = VwSweep(vw)
inputs = [r'C:\data\sweep\1.json', r'C:\data\sweep\2.json']

In [ ]:
opts1 = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume', '-l': 0.1}
opts2 = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume', '-l': 0.01}
opts3 = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume', '-l': 0.001}

opts = [opts1, opts2]
results = sweep.iteration(inputs, opts)

print('Best loss: {0}'.format(results[0].Loss))
print('Best options: {0}'.format(results[0].Opts))

In [ ]:
from VwPipeline.VwOptsGrid import dimension, product, Grid, Configuration
def generate_grid():
    hyper_points_1 = product(
        dimension('-l', [1e-6, 1e-5]),
        dimension('--cb_type', ['ips', 'mtr']),
    )
    hyper_points_2 = dimension('--l1', [1e-09, 1e-07, 1e-5])

    return [
        Grid(hyper_points_1, Configuration(name='Step1', output=1, promote=1)),
        Grid(hyper_points_2, Configuration(name='Step2', output=1, promote=1))
    ]

In [ ]:
grid = generate_grid()
base_command = {'#base': '--ccb_explore_adf --epsilon 0.2 --dsjson --save_resume --preserve_performance_counters'}

result = sweep.run(inputs, grid, base_command)

In [ ]:
result[0].Opts